In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

In [85]:
%load_ext line_profiler

In [1]:
import numpy as np
import cv2

In [100]:
class MOG(object):
  def __init__(self,frame, K=5,T=0.25,lr=0.2):
    self.rows = frame.shape[0]
    self.cols = frame.shape[1]
    self.N = frame.shape[2]
    self.K = K
    self.T = T
    self.dist_mult = 2.5
    self.lr = lr
    self.params = [[[{"mean":np.zeros(frame.shape[2]),"std_dev": 0.25,"weight": (1.0/K)} for a in range(self.K)]  for b in range(self.cols)] for c in range(self.rows)]
    self.HIGH_VAR=2
    self.denom_const = np.power(2*np.pi,(self.N)/2)

  def mahalanaobis_dist_sq(self,pixel,mean,std_dev):
    arg = np.sum((pixel-mean)**2)/std_dev
    return arg
  def prob_pixel(self,sq_dist,std_dev):
    num = np.exp(-0.5*sq_dist)
    denom = self.denom_const*(np.sqrt(self.N)*std_dev)
    return num/denom

  def process_pixel(self,row,col,pixel):
    #sort in decreasing order of weight/std_dev
    sorted(self.params[row][col],key = lambda k: (-1*(k['weight']/k['std_dev'])))
    params = self.params[row][col]

    last_bg_idx = 0
    cum_weight = 0
    for param in params:
      cum_weight += param["weight"]
      if(cum_weight > self.T):
        break
      last_bg_idx += 1
    
    # Estimation Stage
    sq_dist = np.array([self.mahalanaobis_dist_sq(pixel,param["mean"],param["std_dev"]) for param in self.params[row][col]])
    prob = [self.prob_pixel(sq_dist[i],params[i]["std_dev"]) for i in range(len(params))]

    match_idx = len(params)
    for i in range(len(params)):
      if(np.sqrt(sq_dist[i]) < self.dist_mult*params[i]["std_dev"]):
        match_idx = i
        break

    #Updation Stage
    if match_idx==len(params):
      min_idx = np.argmin(prob)
      params[min_idx]["mean"] = pixel
      params[min_idx]["std_dev"] = self.HIGH_VAR
    else:
      for (idx,param) in enumerate(params):
        if idx==match_idx:
          param["weight"] = (1-self.lr)*param["weight"] + self.lr
          rho = self.lr * (prob[match_idx])
          param["mean"] = (1-rho)*param["mean"] + rho*pixel
          variance = (1-rho)*(param["std_dev"]**2) + rho*(np.sum((pixel-param["mean"])**2))
          param["std_dev"] = np.sqrt(variance)
        else:
          param["weight"] = (1-self.lr)*param["weight"]

    return (match_idx <= last_bg_idx)

  def apply(self,frame):
    result = np.zeros(frame.shape[:2],dtype=np.uint8)
    N = frame.shape[2]
    for row in range(frame.shape[0]):
      for col in range(frame.shape[1]):
        result[row][col] = (0 if self.process_pixel(row,col,frame[row][col]) else 255)
    return result

In [101]:
THRESHOLD = 30
CONFIDENCE = 0.1

In [94]:
cap = cv2.VideoCapture('./videos/vtest.avi')

In [93]:
#cap = cv2.VideoCapture(0)

In [91]:
#fgbg = cv2.createBackgroundSubtractorMOG2()

In [95]:
ret, frame = cap.read()
background = np.zeros(frame.shape)
frame = cv2.resize(frame, (100, 50))

In [96]:
fgbg = MOG(frame)

In [99]:
lprun -f MOG.mahalanaobis_dist_sq fgbg.process_pixel(0,0,frame[0][0])

In [82]:
while(1):
    ret, frame = cap.read()
    frame = cv2.resize(frame, (100, 50)) 
    if (not ret):
        break
    fgmask = fgbg.apply(frame)
    cv2.imshow('frame',frame)
    cv2.imshow('output',fgmask)
    
#     bgmask = (fgmask<THRESHOLD)[:,:,None]
#     fgmask = (fgmask>(128 - THRESHOLD))[:,:,None]
#     new_bg = frame*bgmask
#     new_fg = frame*fgmask
#     bg_prob_mask = np.ones(frame.shape) - (bgmask*CONFIDENCE)
#     background = np.asarray((bg_prob_mask*background + (1-bg_prob_mask)*new_bg),np.uint8)
#     foreground = np.asarray(fgmask*new_fg,np.uint8)
#     cv2.imshow('background',background)
#     cv2.imshow('foreground',foreground)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [11]:
params = fgbg.params[30][50]

In [12]:
params

[{'mean': array([  57.,  116.,  109.]),
  'std_dev': 0.17408500218453049,
  'weight': 1.0}]

In [ ]:
[p["weight"] for p in params]

In [ ]:
x = np.zeros(frame.shape[:2],dtype=np.uint8)

In [ ]:
x

In [ ]:
x[0][0]

In [ ]:
x = np.array([3,4,5])

In [ ]:
y = np.array([1,0,3])

In [ ]:
z = np.multiply(x,y)

In [ ]:
np.argmax(z)

In [ ]:
z

In [ ]:
np.argmax(x>6)